In [ ]:
!pip -q install langchain tiktoken replicate

In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.242
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [ ]:
import os

os.environ["REPLICATE_API_TOKEN"] = ""

## LLaMA2 with Replicate


In [ ]:
from langchain.llms import Replicate
from langchain import PromptTemplate, LLMChain

In [ ]:
llm = Replicate(
    model="a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5",
    input={"temperature": 0.75,
           "max_length": 500,
           "top_p": 1},
)


In [ ]:
prompt = """
User: Answer the following yes/no question by reasoning step by step. Can a dog drive a car?
Assistant:
"""
llm(prompt)

'Yes, a dog can drive a car. '

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Replicate(
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model="a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5",
    input={"temperature": 0.75, "max_length": 500, "top_p": 1},
)
prompt = """
User: Answer the following yes/no question by reasoning step by step. Can a dog drive a car?
Assistant:
"""
_ = llm(prompt)

1. Dogs do not have the ability to operate complex machinery like cars.
2. Dogs do not have the physical dexterity or coordination to handle the pedals and steering wheel.
3. Dogs do not have the cognitive ability to understand traffic laws and road signs.

Therefore, the answer is no, a dog cannot drive a car.

Please note that this is a fictional scenario and it is not possible for a dog to drive a car in reality.

## LLaMA70B Model

In [ ]:
llm = Replicate(
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model="replicate/llama70b-v2-chat:2d19859030ff705a87c746f7e96eea03aefb71f166725aee39692f1476566d48",
    input={"temperature": 0.75, "max_length": 500, "top_p": 1},
)
prompt = """
User: Answer the following yes/no question by reasoning step by step. Can a dog drive a car?
Assistant:
"""
_ = llm(prompt)

 No, a dog cannot drive a car.

Explanation:

1. Dogs do not have the physical ability to operate a vehicle, as they do not have opposable thumbs or the necessary strength to manipulate the controls.
2. Dogs also lack the cognitive abilities to understand the complex processes involved in driving a car, such as reading traffic signals, navigating roads, and avoiding obstacles.
3. Additionally, dogs are not licensed or certified to operate vehicles, and it would be unsafe for both the dog and other road users if they were to attempt to drive a car.

Therefore, based on these reasons, it is not possible for a dog to drive a car.

## Prompt

In [ ]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""



def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")



def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.bfloat16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '</s>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [ ]:
instruction = "What is the temperature in Melbourne?"

get_prompt(instruction)

"[INST]<<SYS>>\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n<</SYS>>\n\nWhat is the temperature in Melbourne?[/INST]"

In [ ]:
instruction = "Summarize the following text for me {text}"

system_prompt = "You are an expert and summarization and reducing the number of words used"

get_prompt(instruction, system_prompt)

'[INST]<<SYS>>\nYou are an expert and summarization and reducing the number of words used\n<</SYS>>\n\nSummarize the following text for me {text}[/INST]'

## LangChain basics

In [ ]:
from langchain import PromptTemplate,  LLMChain


In [ ]:
# llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})

In [ ]:
system_prompt = "You are an advanced assistant that excels at translation. "
instruction = "Convert the following text from English to French:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

[INST]<<SYS>>
You are an advanced assistant that excels at translation. 
<</SYS>>

Convert the following text from English to French:

 {text}[/INST]


In [ ]:
text = "how are you today?"
output = llm_chain.run(text)

parse_text(output)

 Sure, I'd be happy to help! The translation of "how are you today?" from English to French is:

" Comment allez-vous aujourd'hui?"

I hope that helps! If you have any other questions or need further assistance, feel free to ask. Sure, I'd be happy to help! The translation of "how are you today?" from English to French is:  "
Comment allez-vous aujourd'hui?"  I hope that helps! If you have any other questions or need further
assistance, feel free to ask.




### Summarization

In [ ]:
instruction = "Summarize the following article for me {text}"
system_prompt = "You are an expert and summarization and expressing key ideas succintly"

template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

[INST]<<SYS>>
You are an expert and summarization and expressing key ideas succintly
<</SYS>>

Summarize the following article for me {text}[/INST]


In [ ]:
def count_words(input_string):
    words = input_string.split(" ")
    return len(words)

text = '''Twitter (now X) CEO Linda Yaccarino claims usage at ‘all time high’ in memo to staff
Twitter’s (now X’s) newly established CEO Linda Yaccarino touts the company’s success and X’s future plans in a company-wide memo obtained by CNBC. The exec once again claims, without sharing any specific metrics, that the service’s usage is at an “all time high,” and hints at what’s to come in terms of new product experiences for the newly rebranded platform.

The service formerly known as Twitter has been working to become more than just a social network and more of an “everything app,” as owner Elon Musk dubbed it.

As the Telsa and Space X exec explained in October 2022, telegraphing Twitter’s eventual rebranding, buying Twitter was meant to be “an accelerant to creating X, the everything app.”


His grand plan has been to create an app that allows creators to monetize their content, then later moves into payments services and even banking, Musk remarked during a Twitter Spaces livestream with advertisers in November. At the time, he even mentioned the possibility of establishing money market accounts on Twitter that would pay a high-interest rate to attract consumers to X.

Those possible product concepts were again referenced in Yaccarino’s new missive, when she writes, “Our usage is at an all time high and we’ll continue to delight our entire community with new experiences in audio, video, messaging, payments, banking – creating a global marketplace for ideas, goods, services, and opportunities.”

Twitter, now X, has already implemented some of Musk’s ideas around videos and creator monetization. In May, the company began allowing subscribers to upload two-hour videos to its service, which advertiser Apple then leveraged when it released the entire first episode of its hit Apple TV+ show “Silo” on the platform. Fired Fox News host Tucker Carlson had been posting lengthy videos to Twitter as well, until ordered to stop by the network.

In addition, earlier this month, Twitter began sharing ad revenue with verified creators.

However, all is not well at Twitter X, whose traffic — at least by third-party measurements — has been dropping. Data from web analytics firm Similarweb indicated Twitter’s web traffic declined 5% for the first two days its latest rival, Instagram Threads, became generally available, compared with the week prior. Plus, Similarweb said Twitter’s web traffic was down 11% compared with the same days in 2022. Additionally, Cloudflare CEO Matthew Prince earlier this month tweeted a graph of traffic to the Twitter.com domain that showed “Twitter traffic tanking,” he said.


Yaccarino subtly pushed back at those reports at the time, claiming that Twitter had its largest usage day since February in early July. She did not share any specific metrics or data. At the same time, however, the company was quietly blocking links to Threads.net in Twitter searches, suggesting it was concerned about the new competition.

Today, Yaccarino repeats her vague claims around X’s high usage in her company-wide memo even as analysts at Forrester are predicting X will either shut down or be acquired within the next 12 months and numerous critics concur that the X rebrand is destined to fail.

Yaccarino’s memo, otherwise, was mostly a lot of cheerleading, applauding X’s team for their work and touting X’s ability to “impress the world all over again,” as Twitter once did.

The full memo, courtesy of CBNC, is below:

Hi team,

What a momentous weekend. As I said yesterday, it’s extremely rare, whether it’s in life or in business, that you have the opportunity to make another big impression. That’s what we’re experiencing together, in real time. Take a moment to put it all into perspective.

17 years ago, Twitter made a lasting imprint on the world. The platform changed the speed at which people accessed information. It created a new dynamic for how people communicated, debated, and responded to things happening in the world. Twitter introduced a new way for people, public figures, and brands to build long lasting relationships. In one way or another, everyone here is a driving force in that change. But equally all our users and partners constantly challenged us to dream bigger, to innovate faster, and to fulfill our great potential.

With X we will go even further to transform the global town square — and impress the world all over again.

Our company uniquely has the drive to make this possible. Many companies say they want to move fast — but we enjoy moving at the speed of light, and when we do, that’s X. At our core, we have an inventor mindset — constantly learning, testing out new approaches, changing to get it right and ultimately succeeding.

With X, we serve our entire community of users and customers by working tirelessly to preserve free expression and choice, create limitless interactivity, and create a marketplace that enables the economic success of all its participants.

The best news is we’re well underway. Everyone should be proud of the pace of innovation over the last nine months — from long form content, to creator monetization, and tremendous advancements in brand safety protections. Our usage is at an all time high and we’ll continue to delight our entire community with new experiences in audio, video, messaging, payments, banking – creating a global marketplace for ideas, goods, services, and opportunities.

Please don’t take this moment for granted. You’re writing history, and there’s no limit to our transformation. And everyone, is invited to build X with us.

Elon and I will be working across every team and partner to bring X to the world. That includes keeping our entire community up to date, ensuring that we all have the information we need to move forward.

Now, let’s go make that next big impression on the world, together.

Linda'''

count_words(text)

940

In [ ]:
output = llm_chain.run(text)
print(count_words(output))
parse_text(output)

 Twitter's CEO Linda Yaccarino claims that the platform's usage is at an all-time high, despite reports of declining traffic. She mentions new product experiences for the recently rebranded platform, including audio, video, messaging, payments, and banking. The company aims to create a global marketplace for ideas, goods, services, and opportunities. Yaccarino also highlights the company's efforts to preserve free expression and choice, create limitless interactivity, and ensure brand safety protections. She encourages the team to continue innovating and invites everyone to build X with them.86
 Twitter's CEO Linda Yaccarino claims that the platform's usage is at an all-time high, despite
reports of declining traffic. She mentions new product experiences for the recently rebranded
platform, including audio, video, messaging, payments, and banking. The company aims to create a
global marketplace for ideas, goods, services, and opportunities. Yaccarino also highlights the
company's effor

## Simple Chatbot

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate

In [ ]:
instruction = "Chat History:\n\n{chat_history} \n\nUser: {user_input} \n\nAI:"
system_prompt = "You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context"

template = get_prompt(instruction, system_prompt)
print(template)

[INST]<<SYS>>
You are a helpful assistant, you always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

{chat_history} 

User: {user_input} 

AI:[/INST]


In [ ]:

prompt = PromptTemplate(
    input_variables=["chat_history", "user_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=False,
    memory=memory,
)

In [ ]:
%%time
llm_chain.predict(user_input="Hi, my name is Sam")

 Hello Sam, it's nice to meet you. How can I assist you today?CPU times: user 308 ms, sys: 14.3 ms, total: 323 ms
Wall time: 5.53 s


" Hello Sam, it's nice to meet you. How can I assist you today?"

In [ ]:
%%time
llm_chain.predict(user_input="Can you tell me about yourself.")

 Sure, I'm just an AI designed to assist with a wide range of tasks, from answering questions to providing information and guidance. I'm here to help you in any way I can, while ensuring that my responses are socially unbiased and positive in nature. Is there something specific you'd like to know or discuss?CPU times: user 288 ms, sys: 15.9 ms, total: 304 ms
Wall time: 14.3 s


" Sure, I'm just an AI designed to assist with a wide range of tasks, from answering questions to providing information and guidance. I'm here to help you in any way I can, while ensuring that my responses are socially unbiased and positive in nature. Is there something specific you'd like to know or discuss?"

In [ ]:
%%time
llm_chain.predict(user_input="Today is Friday. What number day of the week is that?")

 Sure, I can help you with that! Today is indeed Friday, and Friday is the fifth day of the week. So, the answer to your question is "5".CPU times: user 157 ms, sys: 21.2 ms, total: 178 ms
Wall time: 7.96 s


' Sure, I can help you with that! Today is indeed Friday, and Friday is the fifth day of the week. So, the answer to your question is "5".'

In [ ]:
%%time
llm_chain.predict(user_input="what is the day today?")

 Hello! Today is Saturday.CPU times: user 73.4 ms, sys: 5.08 ms, total: 78.5 ms
Wall time: 4.08 s


' Hello! Today is Saturday.'

In [ ]:
%%time
llm_chain.predict(user_input="What is my name?")

 Hello! I'm not able to determine your name as I don't have access to personal information. However, I can tell you that it's great to meet you! Is there anything else I can assist you with?CPU times: user 210 ms, sys: 21.6 ms, total: 232 ms
Wall time: 10.3 s


" Hello! I'm not able to determine your name as I don't have access to personal information. However, I can tell you that it's great to meet you! Is there anything else I can assist you with?"

In [ ]:
%%time
llm_chain.predict(user_input="Can you tell me about the olympics")

 Hello! The Olympics are an international multi-sport event held every four years, where athletes from around the world compete in various sports. The Olympic Games have a rich history, dating back to ancient Greece, and have evolved over time to include a wide range of sports and events.

The modern Olympic Games were revived in 1896, and have been held every four years since then, except during the two World Wars. The Olympics are organized by the International Olympic Committee (IOC), which is responsible for setting the rules and regulations for the games.

The Olympics feature a wide range of sports, including track and field, swimming, gymnastics, basketball, soccer, and many others. Athletes from around the world compete in these events, and medals are awarded to the top three finishers in each event.

The Olympics also have a rich tradition of symbolism and pageantry, with the opening and closing ceremonies featuring elaborate performances and displays of national pride. The Ol

' Hello! The Olympics are an international multi-sport event held every four years, where athletes from around the world compete in various sports. The Olympic Games have a rich history, dating back to ancient Greece, and have evolved over time to include a wide range of sports and events.\n\nThe modern Olympic Games were revived in 1896, and have been held every four years since then, except during the two World Wars. The Olympics are organized by the International Olympic Committee (IOC), which is responsible for setting the rules and regulations for the games.\n\nThe Olympics feature a wide range of sports, including track and field, swimming, gymnastics, basketball, soccer, and many others. Athletes from around the world compete in these events, and medals are awarded to the top three finishers in each event.\n\nThe Olympics also have a rich tradition of symbolism and pageantry, with the opening and closing ceremonies featuring elaborate performances and displays of national pride.

In [ ]:
%%time
llm_chain.predict(user_input="What have we talked about in this Chat?")

 Sure, I can help you with that! In this chat, we've talked about the following topics:

1. Introduction and greeting
2. The day of the week (Friday)
3. Personal information (name)
4. The Olympics
5. Recap of the chat history

Is there anything else I can help you with?CPU times: user 287 ms, sys: 34.2 ms, total: 321 ms
Wall time: 16.3 s


" Sure, I can help you with that! In this chat, we've talked about the following topics:\n\n1. Introduction and greeting\n2. The day of the week (Friday)\n3. Personal information (name)\n4. The Olympics\n5. Recap of the chat history\n\nIs there anything else I can help you with?"